# 02 — Hand-offs entre Agentes

**Objetivo**: Implementar transferencia de control entre agentes cuando uno no puede responder, con protocolo estructurado y limite de hand-offs.

## Contenido
1. Protocolo HandoffDecision
2. Agentes con capacidad de hand-off
3. Limite de hand-offs (max 3)
4. Trazabilidad de transferencias

In [ ]:
import os
import json
import time
from typing import TypedDict, Literal
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage, SystemMessage
from pydantic import BaseModel, Field

load_dotenv()

llm = ChatOpenAI(model="gpt-5-mini", temperature=0)

print("=" * 60)
print("HAND-OFFS ENTRE AGENTES")
print("=" * 60)

In [ ]:
# ============================================================
# PROTOCOLO DE HAND-OFF
# ============================================================

class HandoffDecision(BaseModel):
    """Decision de hand-off entre agentes."""
    puede_responder: bool = Field(description="El agente actual puede responder la pregunta")
    respuesta: str | None = Field(description="Respuesta si puede responder, None si no", default=None)
    target_agent: str | None = Field(description="Agente destino si hace hand-off", default=None)
    reason: str = Field(description="Razon de la decision")
    context_summary: str = Field(description="Resumen del contexto para el siguiente agente")


class AgentSpec:
    """Especificacion de un agente."""
    def __init__(self, name: str, expertise: str, system_prompt: str):
        self.name = name
        self.expertise = expertise
        self.system_prompt = system_prompt


# Definir agentes
agentes = {
    "batman_expert": AgentSpec(
        name="batman_expert",
        expertise="Batman, Gotham City, villanos de DC, Liga de la Justicia",
        system_prompt="Eres un experto en Batman y el universo DC. Solo responde preguntas sobre Batman. Si la pregunta es sobre otro personaje, haz hand-off."
    ),
    "spiderman_expert": AgentSpec(
        name="spiderman_expert", 
        expertise="Spider-Man, Nueva York, villanos de Marvel, Avengers",
        system_prompt="Eres un experto en Spider-Man y el universo Marvel. Solo responde preguntas sobre Spider-Man. Si la pregunta es sobre otro personaje, haz hand-off."
    ),
    "comparador": AgentSpec(
        name="comparador",
        expertise="Comparaciones entre personajes de diferentes universos",
        system_prompt="Eres un experto en comparar personajes de comics. Respondes preguntas que involucran multiples personajes."
    ),
}

print("Agentes definidos:")
for name, spec in agentes.items():
    print(f"  - {name}: {spec.expertise}")

In [ ]:
# ============================================================
# SISTEMA DE HAND-OFFS
# ============================================================

handoff_llm = llm.with_structured_output(HandoffDecision)

def ejecutar_agente(query: str, agent_name: str, context: str = "") -> HandoffDecision:
    """Ejecuta un agente y obtiene su decision."""
    spec = agentes[agent_name]
    otros = [f"- {n}: {s.expertise}" for n, s in agentes.items() if n != agent_name]
    
    prompt = f"""{spec.system_prompt}

Si no puedes responder, haz hand-off a uno de estos agentes:
{chr(10).join(otros)}

{'Contexto previo: ' + context if context else ''}

Pregunta: {query}"""
    
    return handoff_llm.invoke(prompt)


def run_with_handoffs(query: str, initial_agent: str = "batman_expert", max_handoffs: int = 3) -> dict:
    """Ejecuta query con soporte de hand-offs."""
    current_agent = initial_agent
    context = ""
    trace = []
    
    for step in range(max_handoffs + 1):
        t0 = time.time()
        decision = ejecutar_agente(query, current_agent, context)
        latencia = (time.time() - t0) * 1000
        
        trace.append({
            "step": step + 1,
            "agent": current_agent,
            "puede_responder": decision.puede_responder,
            "target": decision.target_agent,
            "reason": decision.reason,
            "latencia_ms": round(latencia, 1),
        })
        
        if decision.puede_responder:
            return {
                "query": query,
                "respuesta": decision.respuesta,
                "agente_final": current_agent,
                "handoffs": step,
                "trace": trace,
            }
        
        if decision.target_agent and decision.target_agent in agentes:
            context = decision.context_summary
            current_agent = decision.target_agent
        else:
            return {
                "query": query,
                "respuesta": f"No se pudo resolver. Ultimo agente: {current_agent}. Razon: {decision.reason}",
                "agente_final": current_agent,
                "handoffs": step,
                "trace": trace,
            }
    
    return {
        "query": query,
        "respuesta": "Maximo de hand-offs alcanzado.",
        "agente_final": current_agent,
        "handoffs": max_handoffs,
        "trace": trace,
    }


print("Sistema de hand-offs configurado (max 3)")

In [ ]:
# ============================================================
# PRUEBAS DE HAND-OFFS
# ============================================================

test_queries = [
    ("Que es la Batcueva?", "batman_expert"),       # No deberia necesitar handoff
    ("Como funciona el sentido aracnido?", "batman_expert"),  # Deberia hacer handoff a spiderman
    ("Compara el liderazgo de ambos heroes", "spiderman_expert"),  # Deberia hacer handoff a comparador
]

print("=" * 60)
print("PRUEBAS DE HAND-OFFS")
print("=" * 60)

for query, start_agent in test_queries:
    result = run_with_handoffs(query, initial_agent=start_agent)
    
    print(f"\nQuery: {query}")
    print(f"  Agente inicial: {start_agent}")
    print(f"  Agente final: {result['agente_final']}")
    print(f"  Hand-offs: {result['handoffs']}")
    print(f"  Trace:")
    for t in result["trace"]:
        status = "RESPONDIO" if t["puede_responder"] else f"HANDOFF \u2192 {t['target']}"
        print(f"    [{t['agent']:18s}] {status} ({t['latencia_ms']}ms)")
        print(f"                        Razon: {t['reason']}")
    print(f"  Respuesta: {result['respuesta'][:200] if result['respuesta'] else 'N/A'}...")

## Takeaways

1. **Hand-offs** permiten a los agentes reconocer sus limitaciones y delegar
2. El protocolo `HandoffDecision` estructura la transferencia con razon y contexto
3. El limite de hand-offs (max 3) previene loops infinitos entre agentes
4. El `context_summary` asegura que el agente receptor tenga informacion relevante
5. La trazabilidad muestra exactamente la cadena de decisiones
6. En produccion, agregar metricas de frecuencia de hand-offs para detectar problemas de routing